# Imports

In [20]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import folium
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='silkeh', api_key='GwBAmaelgKfgkrmuL691')

separator = '\t'

# IMDB
## Creation of RDD's and DF's

In [21]:
#TITLE.BASICS
#tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres
titleBasics_data = sc.textFile('D:/Workspace/_DataMining/DataSets/IMDB/titleBasics.tsv')
titleBasics_rdd = titleBasics_data.filter(lambda l: 'primaryTitle' not in l).map(lambda l: l.split(separator))

#create schema for df
tbFields = []
tbFields.append(StructField('tconst', StringType(), True))
tbFields.append(StructField('titleType', StringType(), True))
tbFields.append(StructField('primaryTitle', StringType(), True))
tbFields.append(StructField('originalTitle', StringType(), True))
tbFields.append(StructField('isAdult', StringType(), True))
tbFields.append(StructField('startYear', StringType(), True))
tbFields.append(StructField('endYear', StringType(), True))
tbFields.append(StructField('runtimeMinutes', StringType(), True))
tbFields.append(StructField('genres', StringType(), True))                           

tbSchema = StructType(tbFields)

#create df
titleBasics_df= sqlContext.createDataFrame(titleBasics_rdd, tbSchema)

In [22]:
#TITLE.AKAS
#tconst, ordering, title, region, language, types, attributes, isOriginalTitle
titleAkas_data = sc.textFile('D:/Workspace/_DataMining/DataSets/IMDB/titleAkas.tsv')
titleAkas_rdd = titleAkas_data.filter(lambda l: 'ordering' not in l).map(lambda l: l.split(separator))

#create schema for df
taFields = []
taFields.append(StructField('titleId', StringType(), True))
taFields.append(StructField('ordering', StringType(), True))
taFields.append(StructField('title', StringType(), True))
taFields.append(StructField('region', StringType(), True))
taFields.append(StructField('language', StringType(), True))
taFields.append(StructField('types', StringType(), True))
taFields.append(StructField('attributes', StringType(), True))
taFields.append(StructField('isOriginalTitle', StringType(), True))                       

taSchema = StructType(taFields)

#create df
titleAkas_df= sqlContext.createDataFrame(titleAkas_rdd, taSchema)

In [24]:
#TITLE.PRINCIPALS
#tconst, ordering, nconst, category, job, characters
titlePrincipals_data = sc.textFile('D:/Workspace/_DataMining/DataSets/IMDB/titlePrincipals.tsv')
titlePrincipals_rdd = titlePrincipals_data.filter(lambda l: 'ordering' not in l).map(lambda l: l.split(separator))

#create schema for df
tpFields = []
tpFields.append(StructField('tconst', StringType(), True))
tpFields.append(StructField('ordering', StringType(), True))
tpFields.append(StructField('nconst', StringType(), True))
tpFields.append(StructField('category', StringType(), True))
tpFields.append(StructField('job', StringType(), True))
tpFields.append(StructField('characters', StringType(), True))                     

tpSchema = StructType(tpFields)

#create df
titlePrincipals_df = sqlContext.createDataFrame(titlePrincipals_rdd, tpSchema)

titlePrincipals_df = titlePrincipals_df.filter(titlePrincipals_df.category.contains("actor") | titlePrincipals_df.category.contains("actress"))
titlePrincipals_df = titlePrincipals_df.drop('job').sort('category', ascending=False)
 

titlePrincipals_df.show(2, truncate = True)


+---------+--------+---------+--------+----------+
|   tconst|ordering|   nconst|category|characters|
+---------+--------+---------+--------+----------+
|tt0199280|       7|nm0848894| actress| ["Anita"]|
|tt0199282|       4|nm0036208| actress| ["Meira"]|
+---------+--------+---------+--------+----------+
only showing top 2 rows



In [25]:
#TITLE.RATINGS
#tconst, averageRating, numVotes
titleRatings_data = sc.textFile('D:/Workspace/_DataMining/DataSets/IMDB/titleRatings.tsv')
titleRatings_rdd = titleRatings_data.filter(lambda l: 'averageRating' not in l).map(lambda l: l.split(separator))

#create schema for df
trFields = []
trFields.append(StructField('tconst', StringType(), True))
trFields.append(StructField('averageRating', StringType(), True))
trFields.append(StructField('numVotes', StringType(), True))                    

trSchema = StructType(trFields)

#create df
titleRatings_df= sqlContext.createDataFrame(titleRatings_rdd, trSchema)

titleRatings_df.show(2, truncate = True)


+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.8|    1416|
|tt0000002|          6.4|     167|
+---------+-------------+--------+
only showing top 2 rows



In [26]:
#TITLE.EPISODE
#tconst, parentTconst, seasonNumber, episodeNumber
titleEpisode_data = sc.textFile('D:/Workspace/_DataMining/DataSets/IMDB/titleEpisode.tsv')
titleEpisode_rdd = titleEpisode_data.filter(lambda l: 'seasonNumber' not in l).map(lambda l: l.split(separator))

#create schema for df
teFields = []
teFields.append(StructField('tconst', StringType(), True))
teFields.append(StructField('parentTconst', StringType(), True))
teFields.append(StructField('seasonNumber', StringType(), True))
teFields.append(StructField('episodeNumber', StringType(), True))                    

teSchema = StructType(teFields)

#create df
titleEpisode_df= sqlContext.createDataFrame(titleEpisode_rdd, teSchema)

titleEpisode_df.show(2, truncate = True)


+---------+------------+------------+-------------+
|   tconst|parentTconst|seasonNumber|episodeNumber|
+---------+------------+------------+-------------+
|tt0041951|   tt0041038|           1|            9|
|tt0042816|   tt0989125|           1|           17|
+---------+------------+------------+-------------+
only showing top 2 rows



In [30]:
#NAME.BASICS
#nconst, primaryName, birthYear, deathYear, primaryProfession, knownForTitles
nameBasics_data = sc.textFile('D:/Workspace/_DataMining/DataSets/IMDB/nameBasics.tsv')
nameBasics_rdd = nameBasics_data.filter(lambda l: 'primaryName' not in l).map(lambda l: l.split(separator))

#create schema for df
nmFields = []
nmFields.append(StructField('nconst', StringType(), True))
nmFields.append(StructField('primaryName', StringType(), True))
nmFields.append(StructField('birthYear', StringType(), True))
nmFields.append(StructField('deathYear', StringType(), True))
nmFields.append(StructField('primaryProfession', StringType(), True))
nmFields.append(StructField('knownForTitles', StringType(), True))

nmSchema = StructType(nmFields)

#create df
nameBasics_df= sqlContext.createDataFrame(nameBasics_rdd, nmSchema)

#nameBasics_df.show(1, truncate = True)

nameBasics_df = nameBasics_df.drop('birthYear', 'knownForTitles')
nameBasics_df = nameBasics_df.filter(nameBasics_df.primaryProfession.contains("actor")|nameBasics_df.primaryProfession.contains("actress")).sort('nconst', ascending=True)

nameBasics_df.show(5, truncate = True)

+---------+---------------+---------+--------------------+
|   nconst|    primaryName|deathYear|   primaryProfession|
+---------+---------------+---------+--------------------+
|nm0000001|   Fred Astaire|     1987|soundtrack,actor,...|
|nm0000002|  Lauren Bacall|     2014|  actress,soundtrack|
|nm0000003|Brigitte Bardot|       \N|actress,soundtrac...|
|nm0000004|   John Belushi|     1982|actor,writer,soun...|
|nm0000005| Ingmar Bergman|     2007|writer,director,a...|
+---------+---------------+---------+--------------------+
only showing top 5 rows



## Join title.Basics and title.Akas

In [31]:
#Aliases
tB_df = titleBasics_df.alias('tB_df')
tA_df = titleAkas_df.alias('tA_df')

#filter title.Akas by region and distinct ID
tA_df_Filtered = tA_df.filter(tA_df.region =='US')
tA_df_Distinct = tA_df_Filtered.dropDuplicates(['titleId']).sort('titleId', ascending=True)

#join title.Basics to title.Akas
tA_Join_tB = tA_df_Distinct.join(tB_df, tA_df_Distinct.titleId == tB_df.tconst, how='left').sort('titleId', ascending=True)
tA_Join_tB = tA_Join_tB.filter(tA_Join_tB.isAdult =='0')
tA_Join_tB = tA_Join_tB.filter(tA_Join_tB.titleType.contains("movie") | tA_Join_tB.titleType.contains("tvSeries") | tA_Join_tB.titleType.contains("tvEpisode"))

#drop unnecessary columns
tAtB_reduced = tA_Join_tB.drop('ordering', 'language', 'types', 
                               'attributes', 'isOriginalTitle', 'tconst', 
                               'primaryTitle', 'originalTitle', 'runtimeMinutes', 
                               'genres', 'isAdult')

tAtB_reduced.show(3, truncate = True)

+---------+--------------------+------+---------+---------+-------+
|  titleId|               title|region|titleType|startYear|endYear|
+---------+--------------------+------+---------+---------+-------+
|tt0000009|          Miss Jerry|    US|    movie|     1894|     \N|
|tt0000147|The Corbett-Fitzs...|    US|    movie|     1897|     \N|
|tt0000630|              Hamlet|    US|    movie|     1908|     \N|
+---------+--------------------+------+---------+---------+-------+
only showing top 3 rows



## Join title.Ratings to AkasBasics

In [ ]:
#aliases
AkasBasics = tAtB_reduced.alias('AkasBasics')
Ratings = titleRatings_df.alias('Ratings')

titleABR = AkasBasics.join(Ratings, AkasBasics.titleId == Ratings.tconst, how='left').sort('titleId', ascending=True)

titleABR.show(3, truncate = True)
